In [1]:
import torch
from torch import nn
import numpy as np
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [2]:
def fn(x):
    rosen=0.
    for i in range(x.numel()-1):
        rosen+=100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return rosen

In [3]:
!rm -rf torch_compile_debug
fn_compiled = torch.compile(fn, backend="inductor", 
                       options={'trace.graph_diagram':True,
                                'trace.enabled':True})
x = torch.rand(5, requires_grad=True)
out = fn_compiled(x.to(device)).sum().backward()

[2023-03-22 19:58:18,599] torch._inductor.debug: [WARNING] model__0_forward_1 debug trace: /pytorch-examples/pytorch-graph-optimization/torch_compile_debug/run_2023_03_22_19_58_18_507442-pid_1882/aot_torchinductor/model__0_forward_1.0


Writing FX graph to file: /pytorch-examples/pytorch-graph-optimization/torch_compile_debug/run_2023_03_22_19_58_18_507442-pid_1882/aot_torchinductor/model__0_forward_1.0/graph_diagram.svg
Writing FX graph to file: /pytorch-examples/pytorch-graph-optimization/torch_compile_debug/run_2023_03_22_19_58_18_507442-pid_1882/aot_torchinductor/model__0_backward_2.1/graph_diagram.svg


[2023-03-22 19:58:18,755] torch._inductor.debug: [WARNING] model__0_backward_2 debug trace: /pytorch-examples/pytorch-graph-optimization/torch_compile_debug/run_2023_03_22_19_58_18_507442-pid_1882/aot_torchinductor/model__0_backward_2.1


In [9]:
from torch.fx import passes, symbolic_trace
model = symbolic_trace(fn)

g = passes.graph_drawer.FxGraphDrawer(model, 'fn')
with open("unoptimized_graph1.svg", "wb") as f:
    f.write(g.get_dot_graph().create_svg())
    
print(model.code)




def forward(self, x):
    pow_1 = x ** 2
    mul = 2 * x;  x = None
    add = pow_1 + mul;  pow_1 = mul = None
    add_1 = add + 1;  add = None
    return add_1
    
